In [1]:
import reflex as rx
import tempfile
from embedchain import App

In [3]:
class State(rx.State):
    """The app state"""
    messages: list[dict] = []
    db_path: str = tempfile.mkdtemp()
    pdf_filename: str = ""
    knowledge_base_files: list[str] = []
    user_question: str = ""
    upload_status: str = ""

    def get_app(self):
        return App.from_config(
            config={
                "llm": {"provider": "ollama",
                        "config": {"model": "llama3.2:latest", "max_tokens": 250, "temperature": 0.5,
                        "stream": True, "base_url": 'http://localhost:3000'}},
                "vectordb": {"provider": "chroma", "config": {"dir": self.db_path}},
                "embedder": {"provider": "ollama",
                             "config": {"model": "llama3.2:latest", "base_url": 'http://localhost:3000'}},
            } 
        )
                                

In [5]:
async def handle_upload(self, files: list[rx.UploadFile]):
    """Handle the file upload and processing."""
    if not files:
        self.upload_status = "no file uploaded!"
        return

    file = files[0]
    upload_data = await file.read()
    outfile = rx.get_upload_dir() / file.filename
    self.pdf_filename = file.filename

    #save the file
    with outfile.open("wb") as file_object:
        file_object.write(upload_data)

    #process and add to knowledge base
    app = self.get_app()
    app.add(str(outfile), data_type="pdf_file")
    self.knowledge_base_files.append(self.pdf_filename)

    self.upload_status = f"Processed and added {self.pdf_filename} to knowledge base!"

In [7]:
def chat(self):
    if not self.user_question:
        return
    app = self.get_app()
    self.messages.append({"role": "user", "content": self.user_question})
    response = app.chat(self.user_question)
    self.messages.append({"role": "assistant", "content": response})
    self.user_question = "" 
    
#clear the question after sending
def clear_chat(self):
    self.messages = []

In [9]:
import reflex as rx  

def index():  
    return rx.vstack(  
        # App title  
        rx.heading("Chat with PDF using Llama 3.2"),  

        # App description  
        rx.text("Chat with a PDF using Llama 3.2 running locally with Ollama!"),  

        # Horizontal layout for PDF upload and other sections  
        rx.hstack(  
            # PDF Upload Section  
            rx.vstack(  
                rx.heading("PDF Upload", size="md"),  # Section heading  
                rx.upload(  
                    label="Upload PDF",  # Button label  
                    multiple=False,  # Allow only one file at a time  
                    accept=".pdf",  # Restrict file types to PDF  
                    on_upload=lambda files: State.handle_upload(files),  # Call the `handle_upload` method  
                ),  
                rx.text("Upload a PDF to add it to the knowledge base."),  
            ),  

            # Chat Section  
            rx.vstack(  
                rx.heading("Chat", size="md"),  # Section heading  
                rx.text_area(  
                    placeholder="Ask a question about the uploaded PDF...",  
                    value=State.user_question,  # Bind to the `user_question` state  
                    on_change=lambda value: State.set_user_question(value),  # Update the state on input  
                ),  
                rx.button(  
                    "Send",   
                    on_click=State.chat,  # Call the `chat` method when clicked  
                ),  
                rx.button(  
                    "Clear Chat",   
                    on_click=State.clear_chat,  # Call the `clear_chat` method when clicked  
                ),  
                rx.box(  
                    rx.foreach(  
                        State.messages,  # Iterate over the `messages` list  
                        lambda message: rx.text(  
                            f"{message['role']}: {message['content']}"  
                        ),  
                    ),  
                    style={"maxHeight": "300px", "overflowY": "scroll"},  # Scrollable chat box  
                ),  
            ),  
            align_items="start",  # Align sections to the top  
            spacing="2em",  # Add spacing between sections  
        ),  
        spacing="2em",  # Add spacing between vertical sections  
        padding="2em",  # Add padding around the layout  
    )  

In [11]:
app = rx.App()  
app.add_page(index)  

